# Assignment

## Task 1. Building Signage Detection and Digit Extraction

In [ ]:
# Connect to my Google Drive
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
import numpy as np # Numpy provides various useful functions and operators for scientific computing
import cv2 as cv # OpenCV provides various useful functions for computer vision
import os # Optional
import glob # Optional
import imutils
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow

In [ ]:
path = "drive/MyDrive/Code/comp3007/assignment/task1/building_signage_recogniser/resources"
filenames = glob.glob(os.path.join(path, "*.jpg"))

In [ ]:
def read_image(filename):
    return cv.imread(filename)

In [ ]:
def preprocess_image(image_bgr):
    # Convert image from BGR to gray
    image_gray = cv.cvtColor(image_bgr, cv.COLOR_BGR2GRAY)

    # Invert image colours
    image_inverted = cv.bitwise_not(image_gray)

    # Perform blackhat operation
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (17, 17))
    image_blackhat = cv.morphologyEx(image_inverted, cv.MORPH_BLACKHAT, kernel)

    # Perform opening operation
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (3, 3))
    image_eroded = cv.erode(image_blackhat, kernel, iterations=1)
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (6, 6))
    image_dilated = cv.dilate(image_eroded, kernel, iterations=3)

    # Perform closing operation
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (6, 6))
    image_dilated = cv.dilate(image_dilated, kernel, iterations=3)
    kernel = cv.getStructuringElement(cv.MORPH_RECT, (3, 3))
    image_eroded = cv.erode(image_dilated, kernel, iterations=1)

    # Perform theshold operation
    ret, image_threshold = cv.threshold(image_eroded, 
                                        150, 255, 
                                        cv.THRESH_BINARY)

    return image_threshold, image_blackhat

In [ ]:
def extract_sign(image_treshhold):
    # Find and sort contours
    contours = cv.findContours(image_threshold, cv.RETR_TREE, cv.CHAIN_APPROX_NONE)[0]
    contours_sorted = sorted(contours, key = cv.contourArea, reverse = True)

    bounding_box_sign = None
    # If the sign was found
    if contours_sorted:
        x, y, w, h = cv.boundingRect(contours_sorted[0])
        bounding_box_sign = (x, y, w, h)

    return bounding_box_sign

In [ ]:
def preprocess_sign(image_cropped):
    # Perform theshold operation on cropped image
    ret, image_threshold = cv.threshold(image_cropped, 40, 255, cv.THRESH_BINARY)

    return image_threshold

In [ ]:
def extract_digits(image_threshold):
    # Find and sort contours
    contours = cv.findContours(image_threshold, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)[0]
    contours_sorted = sorted(contours, key = cv.contourArea, reverse = True)

    # Store minimum digit bounding box aspect ratio, 
    # based on manual analysis of digit aspect ratios
    aspect_ratio_min = 0.35
    aspect_ratio_max = 0.8

    bounding_boxes = list()
    # Loop over digits
    for i in range(min(len(contours_sorted), 3)):
        bounding_box = contours_sorted[i]
        # Create digit bounding box
        x, y, w, h = cv.boundingRect(bounding_box)
        aspect_ratio = float(w) / h

        # If bounding box aspect ratio is within specification
        if aspect_ratio_min <= aspect_ratio <= aspect_ratio_max:
            # Append bounding box to list of bounding boxes
            bounding_boxes.append((x, y, w, h))

    return bounding_boxes

In [ ]:
for filename in filenames:
    # Read full image
    image_bgr = read_image(filename)

    # Preprocess full image
    image_threshold, image_blackhat = preprocess_image(image_bgr)

    # Extract sign
    bounding_box_sign = extract_sign(image_threshold)

    # If a sign was detected 
    if bounding_box_sign:
        x = bounding_box_sign[0]
        y = bounding_box_sign[1]
        w = bounding_box_sign[2]
        h = bounding_box_sign[3]

        # Calculate factor to resize bounding box
        dx = w // 10
        dy = h // 10

        # Calculate tight bounding box crop points
        x_start = max(0, x + dx)
        x_end = min(len(image_blackhat[0]), x + w - dx)
        y_start = max(0, y + dy)
        y_end = min(len(image_blackhat), y + h - dy)

        # Crop full image
        image_cropped = image_blackhat[y_start : y_end, x_start : x_end]

        # Preprocess cropped image
        image_threshold = preprocess_sign(image_cropped)

        # Extract digits
        bounding_boxes_digits = extract_digits(image_threshold)

        # Copy full image
        image_bgr_copy = image_bgr.copy()          

        # Annotate loose sign bounding box
        cv.rectangle(image_bgr_copy, 
                     (x - dx, y - dy), 
                     (x + w + dx, y + h + dy), 
                     (0, 0, 255), 
                     1)

        # For each digit bounding box
        for bounding_box_digit in bounding_boxes_digits:
            x = bounding_box_digit[0]
            y = bounding_box_digit[1]
            w = bounding_box_digit[2]
            h = bounding_box_digit[3]

            # Annotate digit bounding box
            cv.rectangle(image_bgr_copy, 
                            (x + x_start, y + y_start), 
                            (x + x_start + w, y + y_start + h), 
                            (0, 0, 255), 
                            1)
    
    # Show annotated image
    cv2_imshow(image_bgr_copy)

Output hidden; open in https://colab.research.google.com to view.